In [58]:
import os

In [59]:
# function to read a file
def read_file(file):
    with open(file, 'r') as f:
        return f.read()

In [60]:
[1, 2, 3, 4, 5][1:]

[2, 3, 4, 5]

In [61]:
def remove_unecessary_info(text: str) -> str:
    lines = text.split("\n")
    
    start_line_index = None
    end_line_index = None
    
    for i, line in enumerate(lines):
      if line == "### Description":
        start_line_index = i
        
      if line == "[UTS: Handbook 2024](https://www.handbook.uts.edu.au/index.html)":
        end_line_index = i
        break
        
    return "\n".join(lines[start_line_index:end_line_index])

In [62]:
test_file = read_file("./subjects_cleaned/markdown/33130.md")

In [63]:
print(remove_unecessary_info(test_file))

### Description

This subject develops the knowledge and skills necessary for problem-solving
and mathematical modelling at an introductory level. Differential calculus is
applied to model situations in science and engineering that involve
oscillations. Integral calculus is used to solve selected problems involving
first- and second-order differential equations, and to calculate areas,
volumes, lengths and other physical quantities. Vectors, matrix multiplication
and determinants are introduced and applied to problem-solving and modelling.
Sequences and series are reviewed and power series introduced where power
series are used to approximate more functions.

### Subject learning objectives (SLOs)

Upon successful completion of this subject students should be able to:

1. | Describe the relevance of mathematics to engineering and science and the role which engineering and science play in the development and evolution of mathematical ideas and methods | 2. | Apply mathematical tools and